<a href="https://colab.research.google.com/github/supercow5555/COMP-4331/blob/master/DeepNude_(PepeNude%2C_DreamTime)_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [ ]:
#@title Clonong repo and install requirements
!git clone https://github.com/private-dreamnet/dreamtime.git
%cd '/content/dreamtime/src/cli/'
f=open('run.py','r').read().replace('create_watermark(nude)','nude');open('run.py','w').write(f)
!pip install git+https://github.com/jeroenmeulenaar/python3-mega.git
!pip install mediafire
!pip install -r /content/dreamtime/src/cli/requirements-ubuntu.txt




## I got link here:
https://gitgud.io/dreamnet/dreamtime#download-mirrors

Also you can find torrent, and manualy upload to Mega



---


HowToDownload from MediaFire

You need copy link from green download button
and pase it in "Direct_Link".
Example domain: http://download813.mediafire.com/...



## This step will take a few minutes  ~25 minutes

In [ ]:
#@title Download checkpoints (pytorch models  *.lib x3)
%cd "/content/dreamtime/src/cli/"
check_path = '/content/dreamtime/src/cli/checkpoints.zip'


MegaNZ_url = "https://mega.nz/#!3ih2jIgB!EcfD4_K_blnjfNfVmqhV4drrsN_xh4gaMsZzSZIw17s" #@param {type:"string"}
Direct_Link = " " #@param {type:"string"}

Download_from = "MegaNZ" #@param ["MegaNZ", "DirectLink"]

from pathlib import Path
if Path(check_path).exists():
  q_delete = input('Already downloaded. Delete zip file? (y)/n')
  if q_delete.lower() == 'y':
    !rm $check_path

if not Path(check_path).exists():
  print('Downloading started')
  if Download_from == 'MegaNZ':
    from mega import Mega
    m = Mega.from_ephemeral()
    m.download_from_url(MegaNZ_url)



  else:
    import urllib
    urllib.request.urlretrieve ( Direct_Link, "checkpoints.zip" )
  print('Finished')
#пиздец я заебался эту хуйню писать. Надеюсь заработает

def unzip(zip_path, dest_path):
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd
  print("Unziped!")

%cd '/content/dreamtime/src/cli'
check_dir_path = '/content/dreamtime/src/cli/checkpoints'

from pathlib import Path
if Path(check_dir_path).exists():
  !mkdir $check_dir_path

unzip('checkpoints.zip', 'checkpoints')

# Photo edit

In [ ]:
#@title Upload Photo

%cd /content/dreamtime/src/cli/

from pathlib import Path
if not Path('photos').exists():
  !mkdir photos

if not Path('temp_photos').exists():
  !mkdir temp_photos

%cd /content/dreamtime/src/cli/temp_photos
print('Choose 1 png file')
target = 'test'
from google.colab import files
uploaded = files.upload()

for name, bytes_ in uploaded.items():
  if name.endswith('.png') or name.endswith('.jpg'):
    target=name
    break
  else:
    print('Only image')

if target == 'test':
  print('Timeout finished. Pls restart and be faster')
else:
  %matplotlib inline
  from IPython.display import display
  from PIL import Image, ImageDraw
  from ipywidgets import widgets, interact
  import matplotlib.pyplot as plt
  import numpy as np

  image = Image.open(target)
  im_w, im_h = image.size
  im_w+=int(im_w/2)
  im_h+=int(im_h/2)
  size = (im_w, im_h)

  if size == (512, 512):
    print('All right')
    image.save('../photos/'+target[:-3]+'png')
  else:
    print('Pls resize image')
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGBA', size, (255, 255, 255, 0))
    background.paste(
        image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
    )
    image = background

    def f(BoxSize, x, y, save=False):
      ed_image = image.copy()
      draw = ImageDraw.Draw(ed_image)
      draw.rectangle([(x, y),
                      (x+BoxSize, y+BoxSize)],
                     outline='red',
                     width=4)
      if save:
        print('SAVED!')
        crop_im = image.copy()
        crop_im = crop_im.crop((x,y,x+BoxSize,y+BoxSize))
        crop_im = crop_im.resize([512,512], Image.ANTIALIAS)
        crop_im.save('../photos/'+target[:-3]+'png')
      display_im = ed_image.copy()
      display_im.thumbnail((700,700), Image.ANTIALIAS)
      display(display_im)

    def on_butt_clicked(event):
      f(w_box.value,w_x.value,w_y.value,save=True)

    w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
    w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
    w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
    butt = widgets.Button(description='Save')
    butt.on_click(on_butt_clicked)

    display(butt)
    interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@title Upload gif

%cd /content/dreamtime/src/cli/

from pathlib import Path
if not Path('gifs').exists():
  !mkdir gifs

%cd gifs
from google.colab import files
files.upload()


# SEND NUDES!

In [ ]:
#@title Image magic
%cd "/content/dreamtime/src/cli"


path='results'
import os
if not os.path.exists(path):
    os.makedirs(path)

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

mypath = '/content/dreamtime/src/cli/photos'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose file:',
  )
  butt = widgets.Button(description='Start')
  def on_butt_clicked(b):
      print(filename_ch.value)
      %cd '/content/dreamtime/src/cli/'
      !python main.py -i "photos/$filename_ch.value" -o "results/$filename_ch.value" --gpu 0


  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload photo at first')


In [ ]:
#@title Gif magic
%cd "/content/dreamtime/src/cli"


path='gifs'
import os
if not os.path.exists(path):
    os.makedirs(path)

path='gif_results'
if not os.path.exists(path):
    os.makedirs(path)

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

mypath = '/content/dreamtime/src/cli/gifs'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose gif:',
  )
  butt = widgets.Button(description='Start')
  def on_butt_clicked(b):
      print(filename_ch.value)
      %cd '/content/dreamtime/src/cli/'
      !python main.py -i "gifs/$filename_ch.value" -o "gif_results/$filename_ch.value" --gpu 0 --gif


  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload gif at first')



#Tools

In [ ]:
!ls /content/dreamtime/src/cli/temp_photos

In [ ]:
#@title Crop
%cd /content/dreamtime/src/cli/temp_photos
target = ' ' #@param {type:"string"}
%matplotlib inline
from IPython.display import display
from PIL import Image, ImageDraw
from ipywidgets import widgets, interact
import matplotlib.pyplot as plt
import numpy as np

image = Image.open(target)
im_w, im_h = image.size
im_w+=int(im_w/2)
im_h+=int(im_h/2)
size = (im_w, im_h)

if size == (512, 512):
  print('All right')
  image.save('../photos/'+target[:-3]+'png')
else:
  print('Pls resize image')
  image.thumbnail(size, Image.ANTIALIAS)
  background = Image.new('RGBA', size, (255, 255, 255, 0))
  background.paste(
      image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
  )
  image = background

  def f(BoxSize, x, y, save=False):
    ed_image = image.copy()
    draw = ImageDraw.Draw(ed_image)
    draw.rectangle([(x, y),
                    (x+BoxSize, y+BoxSize)],
                   outline='red',
                   width=4)
    if save:
      print('SAVED!')
      crop_im = image.copy()
      crop_im = crop_im.crop((x,y,x+BoxSize,y+BoxSize))
      crop_im = crop_im.resize([512,512], Image.ANTIALIAS)
      crop_im.save('../photos/'+target[:-3]+'png')
    display_im = ed_image.copy()
    display_im.thumbnail((700,700), Image.ANTIALIAS)
    display(display_im)

  def on_butt_clicked(event):
    f(w_box.value,w_x.value,w_y.value,save=True)

  w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
  w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
  w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
  butt = widgets.Button(description='Save')
  butt.on_click(on_butt_clicked)

  display(butt)
  interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@title Rotate image
%cd /content/dreamtime/src/cli/temp_photos
target = ' ' #@param {type:"string"}
ro_deg = -90 #@param {type:"integer"}

from PIL import Image
im_rotate= Image.open(target)
im_rotate=im_rotate.rotate(ro_deg,expand=True)
im_rotate.save(target)

/content/dreamtime/src/cli/temp_photos
